In [189]:
# !ln -s ../../data .
# !pip install backtesting

In [198]:
from sqlalchemy import create_engine, inspect
# Create an SQLite engine
engine = create_engine('sqlite:///data/fngu.db')
table_names = inspect(engine).get_table_names()
table_names

['AAPL', 'AMZN', 'BABA', 'GOOGL', 'META', 'MSFT', 'NFLX', 'NVDA', 'TSLA', 'X']

In [201]:
df

,Date,Open,High,Low,Close
9525,2018-09-20,55.060001,55.570000,54.787498,55.007500
9526,2018-09-21,55.195000,55.340000,54.322498,54.415001
9527,2018-09-24,54.205002,55.314999,54.157501,55.197498
9528,2018-09-25,54.937500,55.705002,54.924999,55.547501
9529,2018-09-26,55.250000,55.937500,54.939999,55.105000
...,...,...,...,...,...
11019,2024-08-29,230.100006,232.919998,228.880005,229.789993
11020,2024-08-30,230.190002,230.399994,227.479996,229.000000
11021,2024-09-03,228.550003,229.000000,221.169998,222.770004
11022,2024-09-04,221.660004,221.779999,217.479996,220.850006


In [209]:
df = pd.read_sql('AAPL', con=engine)

df.drop('Adj Close', axis=1, inplace=True)
df.drop('Volume', axis=1, inplace=True)
# df.set_index('Date', inplace=True)
df=df[-1500:-1]
# print(df['Date'].dtype)
# df['datetime'] = pd.to_datetime(df.index)
# df.set_index('datetime', inplace=True)
df.index = pd.to_datetime(df.Date)
print(df.index.dtype)
df

datetime64[ns]


,Date,Open,High,Low,Close
Date,,,,,
2018-09-20,2018-09-20,55.060001,55.570000,54.787498,55.007500
2018-09-21,2018-09-21,55.195000,55.340000,54.322498,54.415001
2018-09-24,2018-09-24,54.205002,55.314999,54.157501,55.197498
2018-09-25,2018-09-25,54.937500,55.705002,54.924999,55.547501
2018-09-26,2018-09-26,55.250000,55.937500,54.939999,55.105000
...,...,...,...,...,...
2024-08-29,2024-08-29,230.100006,232.919998,228.880005,229.789993
2024-08-30,2024-08-30,230.190002,230.399994,227.479996,229.000000
2024-09-03,2024-09-03,228.550003,229.000000,221.169998,222.770004


In [202]:
print(df.index.dtype)
print(type(df.index))

int64
<class 'pandas.core.indexes.range.RangeIndex'>


In [203]:
print(df.Date.dtype)
print(type(df.Date.dtype))
print(type(df.Date))


datetime64[ns]
<class 'numpy.dtypes.DateTime64DType'>
<class 'pandas.core.series.Series'>


In [210]:
import pandas_ta as ta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

class smastrat(Strategy):
    def init(self):
        super().init()
        self.ma1 = self.I(ta.sma, self.data.Close.s, 5)
        self.ma2 = self.I(ta.sma, self.data.Close.s, 20)

    def next(self):
        super().next()
        # Check for crossover
        # If short term MA crosses long term MA, buy
        # If long term MA crosses short term MA, sell
        # You can also use the `crossover` function from the backtesting library
        # to check for crossover directly
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


In [211]:
bt = Backtest(df, smastrat, cash=10000, commission=0.00)
# bt = Backtest(df, smastrat, cash=250, margin=1/30)
bt.run()

# bt.run()


Start                     2018-09-20 00:00:00
End                       2024-09-05 00:00:00
Duration                   2177 days 00:00:00
Exposure Time [%]                   94.529686
Equity Final [$]                 56110.516235
Equity Peak [$]                  59447.586853
Return [%]                         461.105162
Buy & Hold Return [%]              304.272156
Return (Ann.) [%]                   33.635975
Volatility (Ann.) [%]               41.305701
Sharpe Ratio                         0.814318
Sortino Ratio                        1.662414
Calmar Ratio                         1.072252
Max. Drawdown [%]                  -31.369477
Avg. Drawdown [%]                   -4.194562
Max. Drawdown Duration      525 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                     462.869854
Worst Trade [%]                    462.869854
Avg. Trade [%]                    

In [212]:
bt.plot()


ValueError: failed to validate DatetimeTickFormatter(id='p4884', ...).days: expected a value of type str, got ['%d %b', '%a %d'] of type list